In [2]:
from azureml.core import Workspace, Datastore
from azureml.pipeline.core import PublishedPipeline

ws = Workspace.from_config("../config2.json")
print("Workspace {} is loaded".format(ws.name))

Found the config file in: /Users/sleebapaul/AML_MSFT/config2.json
Workspace Auria is loaded


In [3]:
pipeline_id = "25ab721b-1c39-444f-9730-7a045611af65"

In [4]:
# all_pub_pipelines = PublishedPipeline.get_all(ws)

from azureml.core.authentication import InteractiveLoginAuthentication
import requests

cli_auth = InteractiveLoginAuthentication()
aad_token = cli_auth.get_authentication_header()

published_training_pipeline = PublishedPipeline.get(ws, pipeline_id)

training_pipeline_rest_ep = published_training_pipeline.endpoint

print("The published pipeline REST endpoint is {}".format(training_pipeline_rest_ep))

# specify the param when running the pipeline
response = requests.post(training_pipeline_rest_ep,
                         headers=aad_token,
                         json={"ExperimentName": "Auria_Pipeline_SP",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"model_version": 1.0}})
pipeline_run_id = response.json()["Id"]

print("The run ID is {}".format(pipeline_run_id))

The published pipeline REST endpoint is https://eastus.aether.ms/api/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/PipelinesUsabilityStudy/providers/Microsoft.MachineLearningServices/workspaces/Auria/PipelineRuns/PipelineSubmit/25ab721b-1c39-444f-9730-7a045611af65
The run ID is 1569dbf6-2ff7-4a6a-a172-2fd70d0dc6a8


In [5]:
from azureml.core import Experiment
from azureml.pipeline.core import PipelineRun, StepRun, PortDataReference

experiment = Experiment(ws, "Auria_Pipeline_SP")
pipeline_run = PipelineRun(experiment, pipeline_run_id)
pipeline_run.wait_for_completion()

RunId: 1569dbf6-2ff7-4a6a-a172-2fd70d0dc6a8
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/PipelinesUsabilityStudy/providers/Microsoft.MachineLearningServices/workspaces/Auria/experiments/Auria_Pipeline_SP/runs/1569dbf6-2ff7-4a6a-a172-2fd70d0dc6a8
Status: Running
................................................................
Status: Finished
{'runId': '1569dbf6-2ff7-4a6a-a172-2fd70d0dc6a8', 'status': 'Completed', 'startTimeUtc': '2019-08-06T08:02:58.073576Z', 'endTimeUtc': '2019-08-06T08:18:21.949534Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'HTTP', 'azureml.parameters': '{"model_version":"1.0","nodecount":"3"}', 'azureml.pipelineid': '25ab721b-1c39-444f-9730-7a045611af65'}, 'logFiles': {'logs/azureml/stdoutlogs.txt': 'https://auria5226806917.blob.core.windows.net/azureml/ExperimentRun/dcid.1569dbf6-2ff7-4a6a-a172-2fd70d0dc6a8/logs/azureml/stdoutlogs.txt?sv=2018-1

'Finished'

In [11]:
step_run = pipeline_run.find_step_run("Estimator_GPT_generation")[0]
port_data = step_run.get_output_data("generatedHaiku")
port_data.download("../PipelineData/", show_progress=True)
step_run = pipeline_run.find_step_run("text_to_image_step")[0]
port_data = step_run.get_output_data("generatedArt")
port_data.download("../PipelineData/", show_progress=True)
step_run = pipeline_run.find_step_run("Estimator_coloring_step")[0]
port_data = step_run.get_output_data("coloredImage")
port_data.download("../PipelineData/", show_progress=True)

Path already exists. Skipping download for ./new/azureml/c8619e26-18b2-486d-b85e-8b04731ee24c/generatedHaiku/haiku.txt


Downloaded azureml/0447cc33-d82d-448b-a5ed-8c1b125d34d3/generatedArt/0_s_0_g1.png, 1 files out of an estimated total of 5
Downloaded azureml/0447cc33-d82d-448b-a5ed-8c1b125d34d3/generatedArt/0_s_0_a0.png, 2 files out of an estimated total of 5
Downloaded azureml/0447cc33-d82d-448b-a5ed-8c1b125d34d3/generatedArt/0_s_0_g2.png, 3 files out of an estimated total of 5
Downloaded azureml/0447cc33-d82d-448b-a5ed-8c1b125d34d3/generatedArt/0_s_0_g0.png, 4 files out of an estimated total of 5
Downloaded azureml/0447cc33-d82d-448b-a5ed-8c1b125d34d3/generatedArt/0_s_0_a1.png, 5 files out of an estimated total of 5
Downloaded azureml/ab3cc084-8ae4-41fd-8e29-777949982144/coloredImage/style_images_map.json, 1 files out of an estimated total of 2
Downloaded azureml/ab3cc084-8ae4-41fd-8e29-777949982144/coloredImageexample.png, 2 files out of an estimated total of 2


2